#   

# 作業目標:

    1. 藉由固定的 dataset, 來驗證不同loss function
    2. Dataset 的特性跟我們選用的loss function 對accrancy 的影響
    
    
# 作業重點: 
    請分別選用 "MSE", "binary _crossentropy"
    查看Train/test accurancy and loss rate
    

In [0]:
import keras
from keras.datasets import cifar10
from keras.utils import np_utils
import numpy as np
import logging
import tensorflow as tf

tf.get_logger().setLevel(logging.ERROR)
np.random.seed(10)

Using TensorFlow backend.


In [0]:
# 載入 cifar10 資料集
(x_img_train, y_label_train), (x_img_test, y_label_test) = cifar10.load_data()

# 查詢檔案維度資訊
print(f"Shape of x_img_train: {x_img_train.shape}")
print(f"Shape of y_label_train: {y_label_train.shape}")
print(f"Shape of x_img_test: {x_img_test.shape}")
print(f"Shape of y_label_test: {y_label_test.shape}")

170500096/170498071 [==============================] - 6s 0us/step
Shape of x_img_train: (50000, 32, 32, 3)
Shape of y_label_train: (50000, 1)
Shape of x_img_test: (10000, 32, 32, 3)
Shape of y_label_test: (10000, 1)


In [0]:
# 正規化
x_img_train_normalize = x_img_train.astype('float32') / 255.0
x_img_test_normalize = x_img_test.astype('float32') / 255.0
print(f"Before normalized: shape = {x_img_train.shape} \nfirst pixle: {x_img_train[0][0][0]}")
print(f"After normalized: shape = {x_img_train_normalize.shape} \nfirst pixle: {x_img_train_normalize[0][0][0]}\n")

# OneHot Encoding
y_label_train_OneHot = np_utils.to_categorical(y_label_train)
y_label_test_OneHot = np_utils.to_categorical(y_label_test)
print(f"Before OneHot Encodeing: shape = {y_label_train.shape} \nfirst data: {y_label_train[0]}")
print(f"After OneHot Encodeing: shape = {y_label_train_OneHot.shape} \nfirst data: {y_label_train_OneHot[0]}")

Before normalized: shape = (50000, 32, 32, 3) 
first pixle: [59 62 63]
After normalized: shape = (50000, 32, 32, 3) 
first pixle: [0.23137255 0.24313726 0.24705882]

Before OneHot Encodeing: shape = (50000, 1) 
first data: [6]
After OneHot Encodeing: shape = (50000, 10) 
first data: [0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]


In [0]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D

In [0]:
# 宣告採用序列模型
model = Sequential()

# 建立卷積層 1~4 與池化層 1~4
model.add(Conv2D(filters=32, kernel_size=(3, 3), input_shape=(32, 32, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# 建立神經網路(平坦層、隱藏層、輸出層)
model.add(Flatten())

# 建立全網路連接層
model.add(Dense(512, activation='relu'))
model.add(Dense(512, activation='relu'))

# 建立輸出層
model.add(Dense(10, activation='softmax'))

print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 4, 4, 256)        

In [0]:
# 模型編譯
loss_funs = ['categorical_crossentropy', 'binary_crossentropy', 'mean_squared_error']

for loss_fun in loss_funs:
    model.compile(loss=loss_fun, optimizer='Adam', metrics=['categorical_accuracy'])
    train_history = model.fit(x_img_train_normalize, y_label_train_OneHot, validation_split=0.25, epochs=10, batch_size=256, verbose=0)
    scores = model.evaluate(x_img_test_normalize, y_label_test_OneHot, verbose=0)
    print(f"{loss_fun} - loss: {scores[0]:.5f} accuracy: {scores[1]:.5f}")

categorical_crossentropy - loss: 1.83316 accuracy: 0.70940
binary_crossentropy - loss: 0.30651 accuracy: 0.71770
mean_squared_error - loss: 0.05064 accuracy: 0.70800
